In [1]:
import subprocess
import time
from rra_tools.shell_tools import mkdir # type: ignore
from idd_forecast_mbp import constants as rfc

cause_map = rfc.cause_map
measure_map = rfc.measure_map
ssp_scenarios = rfc.ssp_scenarios

run_date = "2025_07_08"


commands = []
for cause in cause_map:
    for ssp_scenario in ssp_scenarios:
        scenario = ssp_scenarios[ssp_scenario]["dhs_scenario"]
        for measure in measure_map:
            measure_id = measure_map[measure]["measure_id"]
            cause_id = cause_map[cause]["cause_id"]
            command = f"/mnt/team/fhs/pub/venv/fhs_save_results /mnt/team/idd/pub/forecast-mbp/05-upload_data/fhs_upload_folders/cause_id_{cause_id}_measure_id_{measure_id}_scenario_{scenario}_{run_date} --has-past-data False"
            commands.append(command)

In [2]:

# Store results in a list of dictionaries
results = []

# Execute commands sequentially and save results
for i, command in enumerate(commands):
    print(f"Running command {i+1}/{len(commands)}: {command}")
    
    result_data = {
        'command': command,
        'index': i,
        'success': False,
        'stdout': '',
        'stderr': '',
        'return_code': None
    }
    
    try:
        result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
        print(f"Command {i+1} completed successfully")
        
        result_data['success'] = True
        result_data['stdout'] = result.stdout
        result_data['stderr'] = result.stderr
        result_data['return_code'] = result.returncode
        
        if result.stdout:
            print(f"Output: {result.stdout}")
            
    except subprocess.CalledProcessError as e:
        print(f"Command {i+1} failed with error: {e}")
        print(f"Error output: {e.stderr}")
        
        result_data['success'] = False
        result_data['stdout'] = e.stdout if e.stdout else ''
        result_data['stderr'] = e.stderr if e.stderr else ''
        result_data['return_code'] = e.returncode
    
    results.append(result_data)
    time.sleep(1)

print("All commands completed!")

# Now you can work with the results
# Example: Get all successful outputs
successful_outputs = [r['stdout'] for r in results if r['success']]

# Example: Get all failed commands
failed_commands = [r['command'] for r in results if not r['success']]

# Example: Process specific outputs
for result in results:
    if result['success'] and 'specific_text' in result['stdout']:
        # Do something with this specific output
        processed_output = result['stdout'].strip().upper()
        # Further processing...

Running command 1/12: /mnt/team/fhs/pub/venv/fhs_save_results /mnt/team/idd/pub/forecast-mbp/05-upload_data/fhs_upload_folders/cause_id_345_measure_id_1_scenario_66_2025_07_08 --has-past-data False
Command 1 completed successfully
Output: [2025-07-09 10:04:21 PDT] [INFO] [fhs_pipeline_save_results.cli.console.main : 2.0.22] - Beginning FHS Save Results
[2025-07-09 10:04:21 PDT] [WARNING] [fhs_lib_file_interface.lib.fs_implementations.wala_file_system.__init__ : 3.1.21] - WALAFileSystem is not using any remap functions. It will operate on exact FileSystemEntity objects that it is given.
[2025-07-09 10:04:21 PDT] [INFO] [fhs_pipeline_save_results.lib.etl.main : 2.0.22] - Identified 1 files to upload.
[2025-07-09 10:04:21 PDT] [INFO] [fhs_pipeline_save_results.lib.etl.main : 2.0.22] - Beginning upload for draws.h5 (#1/1)
[2025-07-09 10:04:21 PDT] [INFO] [fhs_pipeline_save_results.lib.etl.etl_one_file : 2.0.22] - Reading draws.h5.
[2025-07-09 10:04:29 PDT] [INFO] [fhs_pipeline_save_results

In [3]:
# Parse each output by line and extract the last line
parsed_outputs = []
save_lines = []
for output in successful_outputs:
    lines = output.strip().split('\n')
    last_line = lines[-1] if lines else ""
    parsed_outputs.append({
        'full_output': output,
        'lines': lines,
        'last_line': last_line
    })
    parts = last_line.split("Files saved to:")
    save_lines.append("Files saved to:" + parts[-1].strip())

# Get just the last lines
last_lines = [parsed['last_line'] for parsed in parsed_outputs]

# Print the save_lines
for line in save_lines:
    print(line)

Files saved to:/ihme/forecasting/data/9/future/death/FHS_save_results_2025_07_09_at_10_04_21_scenario_66_malaria
Files saved to:/ihme/forecasting/data/9/future/incidence/FHS_save_results_2025_07_09_at_10_04_51_scenario_66_malaria
Files saved to:/ihme/forecasting/data/9/future/death/FHS_save_results_2025_07_09_at_10_05_20_scenario_0_malaria
Files saved to:/ihme/forecasting/data/9/future/incidence/FHS_save_results_2025_07_09_at_10_05_48_scenario_0_malaria
Files saved to:/ihme/forecasting/data/9/future/death/FHS_save_results_2025_07_09_at_10_06_14_scenario_54_malaria
Files saved to:/ihme/forecasting/data/9/future/incidence/FHS_save_results_2025_07_09_at_10_06_41_scenario_54_malaria
Files saved to:/ihme/forecasting/data/9/future/death/FHS_save_results_2025_07_09_at_10_07_08_scenario_66_ntd_dengue
Files saved to:/ihme/forecasting/data/9/future/incidence/FHS_save_results_2025_07_09_at_10_07_38_scenario_66_ntd_dengue
Files saved to:/ihme/forecasting/data/9/future/death/FHS_save_results_2025_0